# Tokenizer

In [1]:
import json
import nltk
from nltk import word_tokenize
import re

In [2]:
data = []
with open('dump_small_clean.jsonln', 'r', encoding="utf8") as file:
    for line in file:
        data.append(json.loads(line))

In [3]:
def pickLowercaseWords(tokens):
    return [token.lower() for token in tokens if re.fullmatch('\w+', token)]

In [4]:
from tqdm import tqdm

all_words = []
for item in tqdm(data):
    texto = item['body']
    tokens = word_tokenize(texto)
    tokens = pickLowercaseWords(tokens)
    all_words += tokens

100%|██████████████████████████████████████████████████████████████████████████| 11225/11225 [00:10<00:00, 1067.76it/s]


In [5]:
len(all_words)

942514

In [6]:
def removeStopwords(word_list):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    stopwords += nltk.corpus.stopwords.words('english')
    return [i for i in word_list if i not in stopwords]
# stopwords(all_words_clean)

In [7]:
from collections import Counter
word_counts = Counter(removeStopwords(all_words))

In [8]:
word_counts_list = list(word_counts.items())

In [9]:
word_counts_list_sorted = sorted(word_counts_list, key=lambda x: (-x[1], x[0]))

In [10]:
vocab = word_counts_list_sorted[:10000] #10000 mais frequentes

In [11]:
vocab = dict(vocab) #Vocabulario de palavras

In [12]:
vocab

{'categoria': 27626,
 'filmes': 2007,
 'estados': 1874,
 'brasil': 1804,
 'ser': 1675,
 'unidos': 1604,
 'jogos': 1314,
 'futebolistas': 1287,
 'estado': 1240,
 'futebol': 1206,
 'rio': 1193,
 'naturais': 1190,
 'ligações': 1166,
 'portugal': 1145,
 'sobre': 1116,
 'anos': 1111,
 'cidade': 1090,
 'grande': 988,
 'paulo': 939,
 'álbuns': 933,
 'século': 903,
 'olímpicos': 899,
 'universidade': 882,
 'onde': 870,
 'história': 865,
 'janeiro': 859,
 'ver': 846,
 'pode': 819,
 'parte': 786,
 'nova': 785,
 'sul': 776,
 'nome': 758,
 'dois': 753,
 'durante': 724,
 'família': 720,
 'primeira': 718,
 'outros': 695,
 'televisão': 670,
 'após': 668,
 'externas': 660,
 'distrito': 659,
 'ano': 648,
 'sendo': 643,
 'nacional': 624,
 'região': 621,
 'língua': 620,
 'primeiro': 610,
 'clube': 602,
 'ainda': 601,
 'pessoas': 596,
 'verão': 593,
 'comunas': 588,
 'guerra': 582,
 'grupo': 581,
 'tempo': 577,
 'forma': 573,
 'municípios': 556,
 'apenas': 529,
 'três': 522,
 'mundo': 521,
 'desde': 514,


In [13]:
LOWERCASE = [chr(x) for x in range(ord('a'), ord('z') + 1)]
LOWERCASE_OTHERS = ['ç', 'á', 'â', 'é', 'è', 'ó', 'ò', 'í', 'ì', 'ú', 'ù', ]  # etc.
LETTERS = LOWERCASE + LOWERCASE_OTHERS

def edit1(text):
    words = []
    
    # Fase 1: as remoçoes.
    for p in range(len(text)):
        new_word = text[:p] + text[p + 1:]
        if len(new_word) > 0:
            words.append(new_word)
        
    # Fase 2: as adições.
    for p in range(len(text) + 1):
        for c in LETTERS:
            new_word = text[:p] + c + text[p:]
            words.append(new_word)
    
    # Fase 3: as substituições.
    for p in range(len(text)):
        orig_c = text[p]
        for c in LETTERS:
            if orig_c != c:
                new_word = text[:p] + c + text[p + 1:]
                words.append(new_word)
    
    return set(words)

def edit2(text):
    words1 = edit1(text)
    words2 = set()
    for w in words1:
        candidate_words2 = edit1(w)
        candidate_words2 -= words1
        words2.update(candidate_words2)
    words2 -= set([text])
    return words2


In [14]:
# def candidates(word):
#     if word in vocab:
#         candidatos = [word]
#     else:
#         candidatos = []
#     candidatos += \
#         [w for w in edit1(word) if w in vocab] \
#         + [w for w in edit2(word) if w in vocab] \
#         + [word] #Perguntar por que precisa
#     return set(candidatos)

# print(candidates("alexandri"))

# def candidates(word):
#     candidatos = set([word] if word in vocab else [])
#     candidatos.update([w for w in edit1(word) if w in vocab])
#     candidatos.update([w for w in edit2(word) if w in vocab])
#     candidatos.update([word]) #Perguntar por que precisa
#     return set(candidatos)

print(candidates("alexandri"))

def candidates(word):
    candidatos = \
        set([i for i in [word] if i in vocab]) or \
        set([w for w in edit1(word) if w in vocab]) or \
        set([w for w in edit2(word) if w in vocab]) or\
        set([word])
    return candidatos

print(candidates("alexandri"))

NameError: name 'candidates' is not defined

In [ ]:
def P(word, N=sum(vocab.values())): 
        return vocab[word] / N

In [ ]:
def correction(word):
#     correct = ''
#     p = 0
#     for i in candidates(word):
#         if i in vocab:
#             if P(i) > p:
#                 correct = i
#                 p = P(i)
#     return correct
    return max(candidates(word), key=P)

In [ ]:
correction("andré")

In [ ]:
def correction_string(string):
    corrected = ''
    for i in string.split():
        corrected += correction(i) + ' '
        
    return corrected

correction_string('o andré é bacana')

In [ ]:
# qual função dos candidatos usar
# qual função de correção